# Prétraiter le Dataset

In [1]:
from sklearn.datasets import fetch_20newsgroups
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

corpus = [simple_preprocess(doc) for doc in newsgroups.data]
train_corpus, test_corpus = train_test_split(corpus, test_size=0.2, random_state=42)


# Entraîner un Modèle Word2Vec

In [2]:
from gensim.models import Word2Vec

# Entraîner le modèle Word2Vec
word2vec_model = Word2Vec(sentences=train_corpus, vector_size=100, window=5, min_count=5, sg=0)
word2vec_model.save("word2vec_newsgroups.model")

# Vérifier les mots similaires à "computer"
print("Mots similaires à 'computer':")
print(word2vec_model.wv.most_similar('computer'))


Mots similaires à 'computer':
[('network', 0.8505211472511292), ('library', 0.7674833536148071), ('lab', 0.763748049736023), ('graphics', 0.7573402523994446), ('programming', 0.7557602524757385), ('computers', 0.7529143691062927), ('workstation', 0.7475122213363647), ('unix', 0.7448428273200989), ('macintosh', 0.74413001537323), ('platforms', 0.729738175868988)]


### Identifier "l'intrus" dans une liste de mots

In [3]:
# Identifier "l'intrus" parmi une liste de mots
odd_word_out = word2vec_model.wv.doesnt_match(['computer', 'science', 'banana', 'technology'])
print(f"L'intrus est: {odd_word_out}")


L'intrus est: computer


# Entraîner un Modèle Doc2Vec

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Taguer les documents
tagged_docs = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(train_corpus)]

# Entraîner le modèle Doc2Vec
doc2vec_model = Doc2Vec(tagged_docs, vector_size=100, window=5, min_count=5, epochs=20)
doc2vec_model.save("doc2vec_newsgroups.model")

# Accéder au vecteur d'un document
print("Vecteur du premier document:")
print(doc2vec_model.dv[0])


Vecteur du premier document:
[-0.03381013  0.06518947 -0.07866743  0.40570512  0.12915003 -0.02532914
  0.14232537  0.2816578   0.04933567 -0.41299108 -0.07609569 -0.35758916
 -0.18716918 -0.14868917  0.3971536  -0.52370447  0.7227399  -0.05133297
 -0.43979466 -0.31094447  0.0837625  -0.04182419  0.11766963 -0.09440877
  0.42356148 -0.31670442 -0.32585397  0.0785943   0.03274687 -0.20775098
  0.65649104 -0.04857055  0.29470438  0.41676557 -0.20307025  0.15241744
  0.3259403   0.13766386 -0.08078739  0.14999638 -0.19148058 -0.24885008
 -0.3329382  -0.30274275  0.42241356 -0.0147788  -0.0166283  -0.04846225
  0.21012154  0.34886006 -0.15973617 -0.63038415  0.14452069  0.3258041
 -0.02348827  0.42826337  0.08892427 -0.21452388 -0.13584417  0.4110857
  0.4406162   0.2535704   0.11962042 -0.00572437  0.22243166  0.61969256
 -0.26874062  0.25446585 -0.6668966   0.2020142  -0.33793777  0.4477879
  0.358772   -0.14186893 -0.09772515 -0.27306372  0.00302686 -0.34039578
 -0.00272226 -0.4590067  

### Effectuer le Clustering avec les Représentations de Documents


In [5]:
from sklearn.cluster import KMeans
import numpy as np

# Obtenir les embeddings des documents d'entraînement
doc_vectors = np.array([doc2vec_model.dv[str(i)] for i in range(len(tagged_docs))])

# Appliquer KMeans
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(doc_vectors)

# Afficher les documents dans chaque cluster
clusters = {i: [] for i in range(num_clusters)}
for idx, label in enumerate(kmeans.labels_):
    clusters[label].append(idx)

print("Documents par cluster:")
for cluster, docs in clusters.items():
    print(f"Cluster {cluster}: {docs[:5]} (exemples)")


Documents par cluster:
Cluster 0: [9, 11, 13, 17, 19] (exemples)
Cluster 1: [4, 15, 61, 78, 124] (exemples)
Cluster 2: [190, 191, 266, 286, 294] (exemples)
Cluster 3: [0, 1, 2, 3, 6] (exemples)
Cluster 4: [5, 18, 20, 37, 40] (exemples)


### Comparer les Performances de Word2Vec et Doc2Vec

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Exemple: Comparer deux documents avec Doc2Vec
doc1_vec = doc2vec_model.dv[0]
doc2_vec = doc2vec_model.dv[1]
similarity_doc2vec = cosine_similarity([doc1_vec], [doc2_vec])
print(f"Similarité entre deux documents avec Doc2Vec: {similarity_doc2vec[0][0]}")

# Exemple: Comparer deux mots avec Word2Vec
similarity_word2vec = word2vec_model.wv.similarity('computer', 'technology')
print(f"Similarité entre 'computer' et 'technology' avec Word2Vec: {similarity_word2vec}")


Similarité entre deux documents avec Doc2Vec: 0.4206705391407013
Similarité entre 'computer' et 'technology' avec Word2Vec: 0.6803897619247437
